In [4]:
# Load packages
import numpy as np
import xarray as xr
from pathlib import Path
import cftime
import pandas as pd
from matplotlib import pyplot as plt
import os, re

In [26]:
def plev3d(raw_data):
# Info for function
    """
    Function: Compute monthly or seasonal normals for wrfplev3d
    
    Input: raw_data in xarray dataset 
    
    Output: xarray dataset grouped by
    """
# Build empty dictionary
    raw_dataarray={}
# Compute normals by variables
    for var in list(raw_data.data_vars):
# Skip for times
        if 'Times' in var:
            pass
        else:
            raw_dataarray[var]=raw_data[var].mean('time').compute()
# Merge dataarrays back into 1 dataset
    first_loop=True
    for key in raw_dataarray.keys():
        if first_loop:
            output=raw_dataarray[key]
            first_loop=False
        else:
            output=xr.merge([output,raw_dataarray[key]])
    return output

In [27]:
def climate_normals(dir_input,dir_ouput=None):
# Info for function 
    """
    Function:Compute monthly or seasonal normals and create netcdf 
             from monthly average data
                
    Input arguments: directories as strings
    
    Output Directory:<Current working directory>/<pararmeter>/
                     monthly_norm_<start year>/
                     by default unless specified otherwise
    
    Input data: monthly average netcdf with dask
    
    Output data: monthly normals netcdf with dask
    """
# Extract input info
# start year
    start_year=dir_input[-5:-1]
# parameter
    par_start=dir_input.find('wrf/')+4
    par_end=dir_input.find(start_year)-1
    par=dir_input[par_start:par_end]
    print(par,'start\n')
# Create output directory
    if dir_ouput == None:
# Use current working directory by default
        cwd_start=os.getcwd().find('/home')
        cwd=os.getcwd()
        out_dir=os.path.join(cwd[cwd_start::],par+'/monthly_norm_'+start_year)
    else:
        out_dir=os.path.join(dir_ouput,par+'/monthly_norm_'+start_year)
# Check if directory exists
    if os.path.exists(out_dir)==False:
        os.makedirs(out_dir)
    else:
        print('Directory exists\n')
# Open datasets with dask
    raw_data={file.stem :xr.open_dataset(file,chunks={'time':-1},decode_times=False) for file in Path(dir_input).glob('wrfavg/*monthly.nc')}
# Compute monthly normals
    for key in raw_data.keys():
# Check if file exists
        if os.path.isfile(out_dir+'/'+key+'_mon_norm.nc'):
            print(key,'File exists\n')
        elif 'tmp_wrfavg_proc01' in key:
            print(key,'skip for now\n')
        else:
            data=raw_data[key]
            print('\n',key,'start')
# Convert time to datetime64
            data['time'] = pd.date_range(start=start_year+'-01-01', periods=data.sizes['time'], freq='MS')
            data_mon=data.groupby('time.month')
# Compute by dataarray for plev.nc
            if 'plev' in key:
                data_mon=plev3d(data_mon)
            else:
                data_mon=data_mon.mean('time')
# Create netcdf for monthly normals
            data_mon.to_netcdf(out_dir+'/'+key+'_mon_norm.nc')
            print('done\n')


In [22]:
# Run all dirs
dirs=[
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/',
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/',
    '/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2039/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43-flk_2079/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2039/',
    '/scratch/a/aerler/aerler/Mani/wrf/MPIESM12HR_S37_na24_new43loc-flk_2079/',
    '/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43-flk-1979/',
    '/scratch/a/aerler/aerler/Mani/wrf/ERA5_HS_NA24_new43loc-flk-1979/'
    
]

In [23]:
for dir_a in dirs:
    print(dir_a,'\nstart\n')
    climate_normals(dir_a)
    print('\n',dir_a,'\ndone\n')

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/ 
start

MPIESM12HR_S37_na24_conus43-flk start

Directory exists

wrfhydro_d01_monthly File exists

wrflsm_d01_monthly File exists

wrfsrfc_d01_monthly File exists

wrfxtrm_d01_monthly File exists

wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_S37_na24_conus43-flk_2079/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/ 
start

MPIESM12HR_HS_na24_new43loc-flk start

Directory exists

wrfhydro_d01_monthly File exists

wrflsm_d01_monthly File exists

wrfsrfc_d01_monthly File exists

wrfxtrm_d01_monthly File exists

wrfrad_d01_monthly File exists


 /home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43loc-flk_1979/ 
done

/home/p/peltier/mahdinia/scratch/wrf/MPIESM12HR_HS_na24_new43-flk_1979/ 
start

MPIESM12HR_HS_na24_new43-flk start

Directory exists

wrfhydro_d01_monthly File exists

wrfplev3d_d01_monthly skip for now

wrflsm_d01_monthly F